In [ ]:
from os import getcwd
from owlready2 import default_world, onto_path
from pandas import DataFrame, Series, to_datetime, read_csv, concat, Timestamp
from matplotlib.pyplot import subplots, title, show
from numpy import *

# Initialize onotlogy

In [ ]:
CURRENT_PATH = getcwd()
DATABASE_PATH = CURRENT_PATH + "/4WD_car_db.sqlite3"
ONTO_IRI_OLD = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_specific_ontology#"
ONTO_IRI = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_simplified_ontology#"

def get_onto():
    default_world.set_backend(filename=DATABASE_PATH, exclusive=False)
    onto_path.append(CURRENT_PATH)
    return default_world.get_ontology(ONTO_IRI).load()

def close():
    default_world.close()

onto = get_onto()

# Data extracting

In [ ]:
df = DataFrame(columns=['timestamp'])
df.set_index('timestamp',inplace=True)

for measure in onto.search(type = onto.measure):
    res = Series(dtype=object)
    for prop in measure.get_properties():
        for value in prop[measure]:
            if prop.python_name in ['charge','position','samplin_rate']: continue
            res[prop.python_name] = value
    timestamp = res.pop('timestamp')
    df.loc[timestamp,res.keys().values] = res.values
df.reset_index(inplace=True)
df.rename(columns={'index': 'timestamp'}, inplace=True)
df = df.replace({True: 1, False: 0})
print(df.shape)
df.head()

# Updating CSV

In [ ]:
df2 = read_csv('car_data.csv')
df = concat([df2,df], ignore_index=True)
df.to_csv('car_data.csv', index=False)

## Visualization

In [ ]:
THRESHOLD = 1000
df['diff_time'] = df['timestamp'].diff()
df['diff_time'] = df['diff_time'].abs()
df['cycle'] = (df['diff_time'] > THRESHOLD).cumsum()
df['timestamp'] = df['timestamp'] - df.groupby('cycle')['timestamp'].transform('min')
discharge_cycles = df.groupby('cycle')

fig, ax = subplots()

for n, cycle in discharge_cycles:
    ax.plot(cycle.timestamp, cycle.voltage)
    pass

ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
title('Discharge patterns')

show()
discharge_cycles.describe()

In [ ]:
    df['voltage_smoothed'] = df['voltage'].ewm(alpha=0.1, adjust=False).mean()
    df['voltage_smoothed'] = df['voltage_smoothed'].fillna(df['voltage'])
    df.drop(['voltage'], axis=1, inplace=True)

    df_voltage = df['voltage_smoothed']
    df_timestamp = df['timestamp']
    df.drop(['voltage_smoothed','timestamp'], axis=1, inplace=True)
    columns = df.columns

    df = pd.DataFrame(scaler_features.fit_transform(df), columns=columns)
    df_voltage = pd.DataFrame(scaler_voltage.fit_transform(df_voltage.values.reshape(-1,1)), columns=['voltage'])
    df = pd.concat([df_timestamp, df_voltage, df], axis=1)

    THRESHOLD = 1000
    df['diff_time'] = df['timestamp'].diff()
    df['diff_time'] = df['diff_time'].abs()
    df['cycle'] = (df['diff_time'] > THRESHOLD).cumsum()
    df['timestamp'] = df['timestamp'] - df.groupby('cycle')['timestamp'].transform('min')
    discharge_cycles = df.groupby('cycle')

In [ ]:
df['voltage_smoothed'] = df['voltage'].ewm(alpha=0.05, adjust=False).mean()
discharge_cycles = df.groupby('cycle')

fig, ax = subplots()

for n, cycle in discharge_cycles:
    ax.plot(cycle.timestamp, cycle.voltage_smoothed)
    pass

ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
title('Discharge patterns smoothed')
show()

In [ ]:
close()